In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')

import sklearn

## Telemetry feature engineering (4 * 4 = 16 features)

We'd like to predict failure 24 hours in advance. Telemetry data from sensors looked Gaussian, so we'll include mean and standard deviation of telemetry as features. We'll also include min and max values since plots showed spikes in telemetry. Spikes could be correlated with failure, so we'll also include them as features.

We'll also only keep 2015 data to match with other data sets. 

In [2]:
telemetry = pd.read_csv('archive/PdM_telemetry.csv')

In [3]:
def flatten_index_names(names) -> list[str]:
    # Method to flatten multilevel index labels after aggregation
    res = []
    for [top, bottom] in names:
        if not bottom:
            res.append(top)
        else:
            res.append(f"{top}_{bottom}")
    return res

In [4]:
telemetry['datetime'] = pd.to_datetime(telemetry['datetime'])
telemetry['dayofyear'] = telemetry['datetime'].apply(lambda x: x.dayofyear)
telemetry['year'] = telemetry['datetime'].apply(lambda x: x.year)

telemetry_preprocessed = (telemetry.query('year == 2015')
                          .groupby(['dayofyear', 'machineID'])
                          .agg({
                            'volt' : ['min', 'max', 'mean', 'std'],
                            'rotate' : ['min', 'max', 'mean', 'std'],
                            'pressure' : ['min', 'max', 'mean', 'std'],
                            'vibration' : ['min', 'max', 'mean', 'std'],
                            })
                          .reset_index())

# flattens level names
telemetry_preprocessed = pd.DataFrame(data = telemetry_preprocessed.values, columns=flatten_index_names(telemetry_preprocessed.columns))
telemetry_preprocessed

,dayofyear,machineID,volt_min,volt_max,volt_mean,volt_std,rotate_min,rotate_max,rotate_mean,rotate_std,pressure_min,pressure_max,pressure_mean,pressure_std,vibration_min,vibration_max,vibration_mean,vibration_std
0,1.0,1.0,151.335682,182.739113,167.576533,9.300337,346.149335,527.349825,440.515328,49.590263,75.237905,113.077935,98.522345,10.588562,25.990511,51.021486,40.049623,5.739395
1,1.0,2.0,153.540806,194.108213,173.738811,12.067372,369.738792,543.802540,445.200094,41.838707,76.005332,115.061863,100.728570,10.346696,32.160232,50.690561,41.439609,4.650438
2,1.0,3.0,138.898164,185.482043,168.923402,10.809796,382.648588,531.139800,454.152365,48.902810,80.908824,119.853928,100.362017,10.542271,27.495507,44.698218,36.289645,5.179911
3,1.0,4.0,151.152126,199.358844,168.459275,15.793301,327.243866,551.327283,447.758764,51.761877,81.506614,117.850256,98.139800,8.020534,33.566113,54.352090,41.573753,5.078756
4,1.0,5.0,143.414645,206.533090,173.108601,14.704104,308.578855,539.732729,450.183235,62.748653,76.874388,119.082137,97.544686,12.322612,32.356743,46.851727,39.287064,4.131422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36495,365.0,96.0,144.000510,202.861352,167.678923,15.527895,361.359695,505.766217,445.124075,39.431196,81.818280,127.470057,101.446391,10.468468,34.277274,48.426372,41.350466,4.642365
36496,365.0,97.0,149.112150,187.864746,172.083943,9.761519,347.319414,555.031859,462.796362,49.056849,60.695243,105.500723,93.764293,9.386723,31.465229,49.584052,40.879404,5.199864
36497,365.0,98.0,135.603724,192.976814,168.924428,16.373080,359.260699,522.911573,444.468821,46.016142,93.223992,144.636682,120.341052,13.967056,34.666295,45.658639,39.575961,3.265468
36498,365.0,99.0,134.562385,202.373500,172.421048,17.131220,374.956982,542.069837,458.079445,48.980932,81.666934,123.763248,101.562921,9.474167,33.256047,50.949613,40.402514,4.351219


## Errors feature engineering (5 features)
Errors are categorical data, so we'll one-hot encode them using `pd.get_dummies`. 

In [5]:
errors = pd.read_csv('./archive/PdM_errors.csv')
errors

,datetime,machineID,errorID
0,2015-01-03 07:00:00,1,error1
1,2015-01-03 20:00:00,1,error3
2,2015-01-04 06:00:00,1,error5
3,2015-01-10 15:00:00,1,error4
4,2015-01-22 10:00:00,1,error4
...,...,...,...
3914,2015-11-21 08:00:00,100,error2
3915,2015-12-04 02:00:00,100,error1
3916,2015-12-08 06:00:00,100,error2
3917,2015-12-08 06:00:00,100,error3


In [6]:
errors['datetime'] = pd.to_datetime(errors['datetime'])
errors['dayofyear'] = errors['datetime'].apply(lambda x: x.dayofyear)
errors['year'] = errors['datetime'].apply(lambda x: x.year)

errors_preprocessed = (errors.query('year == 2015')
                            .merge(pd.get_dummies(errors['errorID']), left_index=True, right_index=True)
                            .drop('errorID', axis=1)
                            .groupby(['dayofyear', 'machineID'])
                            .agg({
                                f"error{i+1}" : 'max' for i in range(5)
                            })
                            .reset_index())

errors_preprocessed

,dayofyear,machineID,error1,error2,error3,error4,error5
0,1,10,1,0,0,0,0
1,1,14,0,0,0,1,0
2,1,24,1,0,0,0,0
3,1,42,0,1,0,0,0
4,1,43,0,0,1,0,0
...,...,...,...,...,...,...,...
3434,364,95,1,0,0,0,0
3435,365,23,0,0,1,0,0
3436,365,80,0,0,1,0,0
3437,365,91,0,0,0,1,0


## Maintenance feature engineering (4 features)
Maintenance is carried out on components which are also categorical data. We'll similarly one-hot encode them, making sure to only keep maintenance records from 2015. 

In [7]:
maintenance = pd.read_csv('./archive/PdM_maint.csv')
maintenance

,datetime,machineID,comp
0,2014-06-01 06:00:00,1,comp2
1,2014-07-16 06:00:00,1,comp4
2,2014-07-31 06:00:00,1,comp3
3,2014-12-13 06:00:00,1,comp1
4,2015-01-05 06:00:00,1,comp4
...,...,...,...
3281,2015-10-10 06:00:00,100,comp3
3282,2015-10-25 06:00:00,100,comp4
3283,2015-11-09 06:00:00,100,comp4
3284,2015-12-09 06:00:00,100,comp2


In [8]:
maintenance['datetime'] = pd.to_datetime(maintenance['datetime'])
maintenance['dayofyear'] = maintenance['datetime'].apply(lambda x: x.dayofyear)
maintenance['year'] = maintenance['datetime'].apply(lambda x: x.year)

maintenance_preprocessed = (maintenance.query('year == 2015')
                                       .merge(pd.get_dummies(maintenance['comp']), left_index=True, right_index=True)
                                       .drop('comp', axis=1)
                                       .groupby(['dayofyear', 'machineID'])
                                       .agg({
                                           f"comp{i+1}" : 'max' for i in range(4)
                                       })
                                       .reset_index())

maintenance_preprocessed

,dayofyear,machineID,comp1,comp2,comp3,comp4
0,1,14,1,0,0,0
1,1,17,0,0,0,1
2,1,34,1,1,0,0
3,1,51,1,0,1,0
4,1,65,1,0,0,0
...,...,...,...,...,...,...
2151,365,69,0,0,1,0
2152,365,84,1,1,0,0
2153,365,90,1,0,0,0
2154,365,92,0,0,1,0


## Machines feature engineering (1 + 4 = 5 features)
Machine age is a numerical feature and can be kept as is. The model of a machine is categorical, so we'll also one-hot encode it. 

In [9]:
machines = pd.read_csv('archive/PdM_machines.csv')
machines.head()

,machineID,model,age
0,1,model3,18
1,2,model4,7
2,3,model3,8
3,4,model3,7
4,5,model3,2


In [10]:
machines_preprocessed = (machines.merge(pd.get_dummies(machines['model']), 
                                        left_index=True, 
                                        right_index=True)
                                 .drop('model', axis=1))
machines_preprocessed

,machineID,age,model1,model2,model3,model4
0,1,18,0,0,1,0
1,2,7,0,0,0,1
2,3,8,0,0,1,0
3,4,7,0,0,1,0
4,5,2,0,0,1,0
...,...,...,...,...,...,...
95,96,10,0,1,0,0
96,97,14,0,1,0,0
97,98,20,0,1,0,0
98,99,14,1,0,0,0


## Failures feature engineering (1 label)
Since any type of failure results in a machine shutting down, we'll use an indicator variable for the failure. NB: the type of failure could be useful information for a technician to service the machine.

Here we can simplify our problem to a binary classification of failure vs non-failure. This will make proactive maintenance more amenable to simple ML models like decision trees which can give us clear insight into what is causing machine failure. It will also make model training faster (faster results for the client) and model selection simpler (only a handful of interpretable binary classification algorithms). 

In [11]:
failures = pd.read_csv('archive/PdM_failures.csv')
failures['datetime'] = pd.to_datetime(failures['datetime'])
failures['failure'] = 1 # indicator to note that a failure occurred! For now let's ignore the type of failure!

failures

,datetime,machineID,failure
0,2015-01-05 06:00:00,1,1
1,2015-03-06 06:00:00,1,1
2,2015-04-20 06:00:00,1,1
3,2015-06-19 06:00:00,1,1
4,2015-09-02 06:00:00,1,1
...,...,...,...
756,2015-11-29 06:00:00,99,1
757,2015-12-14 06:00:00,99,1
758,2015-02-12 06:00:00,100,1
759,2015-09-10 06:00:00,100,1


### Encode labels by marking a failure as occurring the day before it occured
We want to make sure we predict a failure *before* it happens! We'll shift failures one day before they happen in the label. That way, features that indicate a failure happens tomorrow will match up with the failure label when we join all the `DataFrames`.

This is also quantitatively justifiable at least in the telemetry case since we saw nontrivial autocorrelation up to 24 hours. 

In [12]:
failures['daybefore'] = failures['datetime'] - pd.tseries.offsets.Day()
failures['dayofyear'] = failures['daybefore'].apply(lambda x: x.dayofyear)

# failures_preprocessed = (failures.merge(pd.get_dummies(failures['failure']), left_index=True, right_index=True)
#                                 .rename(columns = {
#                                     f"comp{i+1}" : f"fail{i+1}" for i in range(4)
#                                 })
#                                 .drop(['failure', 'datetime'], axis=1)
#                                 .groupby(['dayofyear', 'machineID'])
#                                 .agg({
#                                     f"fail{i+1}" : 'max' for i in range(4)
#                                 })
#                                 .reset_index())

# failures_preprocessed

failures_preprocessed = (failures.drop(['datetime', 'daybefore'], axis=1))
failures_preprocessed

,machineID,failure,dayofyear
0,1,1,4
1,1,1,64
2,1,1,109
3,1,1,169
4,1,1,244
...,...,...,...
756,99,1,332
757,99,1,347
758,100,1,42
759,100,1,252


## Merging DataFrames
Finally, we'll join telemetry, errors, maintenance, machine, and failure `DataFrames` on the `machineID` and `dayofyear` columns. This final `DataFrame` will be used for model selection. 

In [13]:
df_merged = (telemetry_preprocessed.merge(errors_preprocessed, on=['dayofyear', 'machineID'], how='left')
 .merge(maintenance_preprocessed, how='left', on=['dayofyear', 'machineID'])
 .merge(machines_preprocessed, on='machineID')
 .merge(failures_preprocessed, on=['dayofyear', 'machineID'], how='left')
 .fillna(0))

df_merged.head()

,dayofyear,machineID,volt_min,volt_max,volt_mean,volt_std,rotate_min,rotate_max,rotate_mean,rotate_std,...,comp1,comp2,comp3,comp4,age,model1,model2,model3,model4,failure
0,1.0,1.0,151.335682,182.739113,167.576533,9.300337,346.149335,527.349825,440.515328,49.590263,...,0.0,0.0,0.0,0.0,18,0,0,1,0,0.0
1,2.0,1.0,140.776309,200.872430,169.795758,15.742155,384.645962,519.452812,446.832666,38.800266,...,0.0,0.0,0.0,0.0,18,0,0,1,0,0.0
2,3.0,1.0,154.199258,194.942847,171.862244,11.182853,374.127148,568.972310,459.204742,47.387959,...,0.0,0.0,0.0,0.0,18,0,0,1,0,0.0
3,4.0,1.0,129.016707,215.656488,174.792428,19.224657,365.213804,517.348533,448.743201,34.008026,...,0.0,0.0,0.0,0.0,18,0,0,1,0,1.0
4,5.0,1.0,127.163620,202.520488,171.018408,17.900560,376.719605,575.505189,454.822750,47.803621,...,1.0,0.0,0.0,1.0,18,0,0,1,0,0.0


In [14]:
df_merged.to_csv('PdM_merged.csv')